In [ ]:
import pandas as pd
import matplotlib as plt
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential, load_model # type: ignore
from tensorflow.keras.layers import LSTM, Dense # type: ignore
from tensorflow.keras.optimizers import Adam # type: ignore
from tensorflow.keras.callbacks import ModelCheckpoint # type: ignore
from keras import metrics

In [ ]:
df = pd.read_csv('Data_In.csv')
df.rename(columns = {'company_name' : 'Company'}, inplace = True)

# Print Rows with Null Values
# df[df.isnull().any(axis=1)]

In [ ]:
search_term = 'AMAZON'
factor = 'Company'

# Filter rows where 'Type' is equal to 'Health'
filtered_df = df[df[factor] == search_term]


In [ ]:
# Reshape data into a 1D array
# Scale data to the min and max of the rows i.e 0 -1 range
# Flatten the data once again to a 1D array
# ====================================================================

data = filtered_df['Close'].values.reshape(-1, 1)
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)
data_scaled2 = data_scaled.flatten()


In [ ]:
def create_sequences(data, seq_length):
    sequences, labels = [], []
    
    for i in range(len(data) - seq_length):
        seq = data[i:i + seq_length, 0]
        label = data[i + seq_length, 0]
        sequences.append(seq)
        labels.append(label)
    return np.array(sequences), np.array(labels)

In [ ]:
seq_length = 5
n_samples = len(data_scaled)
sequences, labels = create_sequences(data_scaled, seq_length)
split = int(0.8 * len(sequences))
X_test, y_test = sequences[split:], labels[split:]
X_train, y_train = sequences[:split], labels[:split]

In [ ]:
model_lstm = Sequential()
model_lstm.add(LSTM(100, activation='relu', input_shape=(seq_length, 1), return_sequences=True))
model_lstm.add(LSTM(50, activation='relu'))
model_lstm.add(Dense(32))
model_lstm.add(Dense(1))
model_lstm.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

In [126]:
model_lstm.fit(X_train, y_train, epochs=200, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/200
168/168 ━━━━━━━━━━━━━━━━━━━━ 12s 24ms/step - loss: 0.0024 - val_loss: 6.4990e-04
Epoch 2/200
168/168 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 1.1997e-05 - val_loss: 6.1988e-04
Epoch 3/200
168/168 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - loss: 1.3161e-05 - val_loss: 0.0010
Epoch 4/200
168/168 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - loss: 1.6696e-05 - val_loss: 0.0012
Epoch 5/200
168/168 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - loss: 1.3535e-05 - val_loss: 7.9624e-04
Epoch 6/200
168/168 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - loss: 1.3608e-05 - val_loss: 9.0646e-04
Epoch 7/200
168/168 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - loss: 1.3777e-05 - val_loss: 0.0015
Epoch 8/200
168/168 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - loss: 1.6151e-05 - val_loss: 0.0011
Epoch 9/200
168/168 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - loss: 1.5221e-05 - val_loss: 7.4005e-04
Epoch 10/200
168/168 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - loss: 1.8269e-05 - val_loss: 0.0018
Epoch 11/200
168/168 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - loss:

In [131]:
model_lstm.save('model_lstm_6.keras')

In [132]:
predictor = load_model('model_lstm_6.keras')
amazn = predictor.predict(X_test)

42/42 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step


In [133]:
amazn = scaler.inverse_transform(amazn)
amazn_y = scaler.inverse_transform(y_test.reshape(-1, 1))

In [135]:
dates_test = filtered_df['Date'].iloc[split + seq_length:]
# df_plot_lstm = pd.DataFrame({'Fechas': dates_test, 'Real': y_test_actual_LSTM.flatten(), 'Predict': predictions__LSTM.flatten()})
df_plot_lstm = pd.DataFrame({'Date': dates_test[:len(amazn)], 'Real': amazn_y.flatten(), 'Predict': amazn.flatten()})

df_plot_lstm.tail(50)
# df_plot_lstm.to_csv('AMAZON.csv')